In [6]:
import pandas as pd
import numpy as np
from datetime import datetime

In [7]:
def load_and_examine_data():
    """Load the dataset and perform initial examination"""
    df = pd.read_csv('data/events.csv')
    print("Initial dataset shape:", df.shape)
    print("\nMissing values summary:")
    print(df.isnull().sum())
    return df

In [8]:
# Clean and preprocess the data
def clean_data(df):
    """Clean and preprocess the dataset"""
    # Create a copy to avoid modifying original data
    df_clean = df.copy()
    
    # 1. Handle missing values
    
    # For brand:
    # Replace missing brands with 'unknown_brand' since we can't reliably impute brands
    df_clean['brand'] = df_clean['brand'].fillna('unknown_brand')
    
    # For category_code:
    # First, try to fill missing category_codes based on category_id if possible
    category_mapping = df_clean.groupby('category_id')['category_code'].agg(lambda x: x.mode().iloc[0] if not x.mode().empty else 'unknown')
    df_clean['category_code'] = df_clean.apply(
        lambda row: category_mapping[row['category_id']] 
        if pd.isna(row['category_code']) else row['category_code'],
        axis=1
    )
    
    # Fill any remaining missing category_codes with 'unknown_category'
    df_clean['category_code'] = df_clean['category_code'].fillna('unknown_category')
    
    # For user_session:
    # Generate new session IDs for missing sessions using timestamp and user_id
    df_clean['user_session'] = df_clean['user_session'].fillna(
        df_clean.apply(
            lambda row: f"generated_session_{row['user_id']}_{pd.Timestamp(row['event_time']).strftime('%Y%m%d%H%M%S')}",
            axis=1
        )
    )
    
    # 2. Convert event_time to datetime
    df_clean['event_time'] = pd.to_datetime(df_clean['event_time'])
    
    return df_clean

# Validate the cleaned dataset
def validate_cleaning(df_original, df_cleaned):
    """Validate the cleaning process"""
    validation_results = {
        'original_shape': df_original.shape,
        'cleaned_shape': df_cleaned.shape,
        'missing_values_original': df_original.isnull().sum(),
        'missing_values_cleaned': df_cleaned.isnull().sum(),
        'unique_values_comparison': {
            col: {
                'original': df_original[col].nunique(),
                'cleaned': df_cleaned[col].nunique()
            }
            for col in df_original.columns
        }
    }
    return validation_results

In [9]:
# Load data
print("Loading data...")
df_original = load_and_examine_data()

# Clean data
print("\nCleaning data...")
df_cleaned = clean_data(df_original)

# Validate results
print("\nValidating cleaning results...")
validation = validate_cleaning(df_original, df_cleaned)

Loading data...
Initial dataset shape: (885129, 9)

Missing values summary:
event_time            0
event_type            0
product_id            0
category_id           0
category_code    236219
brand            212364
price                 0
user_id               0
user_session        165
dtype: int64

Cleaning data...

Validating cleaning results...


In [10]:
df_cleaned

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2020-09-24 11:57:06+00:00,view,1996170,2144415922528450000,electronics.telephone,unknown_brand,31.90,1515915625519380000,LJuJVLEjPT
1,2020-09-24 11:57:26+00:00,view,139905,2144415922528450000,computers.components.cooler,zalman,17.16,1515915625519380000,tdicluNnRY
2,2020-09-24 11:57:27+00:00,view,215454,2144415922528450000,computers.components.videocards,unknown_brand,9.81,1515915625513230000,4TMArHtXQy
3,2020-09-24 11:57:33+00:00,view,635807,2144415922528450000,computers.peripherals.printer,pantum,113.81,1515915625519010000,aGFYrNgC08
4,2020-09-24 11:57:36+00:00,view,3658723,2144415922528450000,computers.components.videocards,cameronsino,15.87,1515915625510740000,aa4mmk0kwQ
...,...,...,...,...,...,...,...,...,...
885124,2021-02-28 23:55:01+00:00,view,953226,2144415922528450000,computers.components.videocards,unknown_brand,219.94,1515915625611020000,FRLqIttxKU
885125,2021-02-28 23:58:05+00:00,view,1715907,2144415922528450000,electronics.video.tv,starwind,80.03,1515915625611020000,g6WqPf50Ma
885126,2021-02-28 23:58:09+00:00,view,4170534,2144415922528450000,electronics.clocks,amazfit,64.92,1515915625611020000,xNIJBqZdkd
885127,2021-02-28 23:58:14+00:00,view,888273,2144415922528450000,electronics.telephone,unknown_brand,10.16,1515915625611020000,9pCbKMIcSx


In [11]:
validation

{'original_shape': (885129, 9),
 'cleaned_shape': (885129, 9),
 'missing_values_original': event_time            0
 event_type            0
 product_id            0
 category_id           0
 category_code    236219
 brand            212364
 price                 0
 user_id               0
 user_session        165
 dtype: int64,
 'missing_values_cleaned': event_time       0
 event_type       0
 product_id       0
 category_id      0
 category_code    0
 brand            0
 price            0
 user_id          0
 user_session     0
 dtype: int64,
 'unique_values_comparison': {'event_time': {'original': 845041,
   'cleaned': 845041},
  'event_type': {'original': 3, 'cleaned': 3},
  'product_id': {'original': 53453, 'cleaned': 53453},
  'category_id': {'original': 1, 'cleaned': 1},
  'category_code': {'original': 107, 'cleaned': 107},
  'brand': {'original': 999, 'cleaned': 1000},
  'price': {'original': 12422, 'cleaned': 12422},
  'user_id': {'original': 13353, 'cleaned': 13353},
  'user_

In [12]:
# Save cleaned dataset
df_cleaned.to_csv('data/events_cleaned.csv', index=False)